# How to Tame an LLM using Retrieval Augmented Generation (RAG)

In this lecture:
- The problem of LLM hallucination
- Guide prompts that include reference material
- Retrieval Augmented Generation (RAG)
    - Embedding
    - Vector stores
    - Closest neighbour search

### How do LLMs really work?
- This is a useful thought experiment. ([Sidenote: This is where it all started](https://arxiv.org/abs/1706.03762)).

- Turns out they're fairly similar to us in some regards (long-term & short-term 'memory', ability to '[pay attention](https://arxiv.org/pdf/2307.03172.pdf)').
- Where possible, put LLMs in position to use short-term memory and help them pay attention.
- LLMs are great at [pattern matching](https://arxiv.org/abs/2005.14165) and following syntactic rules.
- There are cases where LLMs provide a solution to a problem, but [they may be suboptimal](https://aclanthology.org/2023.findings-acl.426.pdf).

### Retrieval Augmented Generation (RAG) - An Antidote to Hallucination
- One such way of limiting the use of LLMs to what they are best at.

- Uses [in-context learning](https://arxiv.org/abs/2301.00234) to give the LLM a usable short-term memory.

Let's use this to ask questions about some lecture notes
### Reading my PDF in Python



In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [3]:
from PyPDF2 import PdfReader
reader = PdfReader("Lecture Notes.pdf") # read the pdf file

# Read each page and store them as a string
lecture_notes  = ''.join([page.extract_text() for page in reader.pages]) # create one big string of text

In [3]:
from openai import OpenAI
import os

# Load our OpenAI API key
openai = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY")) # TODO set your API key

### Augmented Generation - RAG's little brother



In [5]:
def ask_query (query, context):

    # Tell the LLM to only use the data we give it
    guide_prompt = # TODO create a guide prompt that asks the LLM to answer the query using the context
    messages = # TODO create a list of messages that ends with the quide prompt

    response = # TODO create a completion using the guide prompt

    response_content = # TODO get the response content from the response

    return(response_content)

# TODO ask the LLM to answer the query using the lecture notes as context

'An objective function is a criterion that is used to compare different designs in an optimization problem. It is expressed as a function of the design variables and represents the goal or objective that the optimization is trying to achieve. The objective function is typically specified based on physical or economic considerations and can vary depending on the specific problem. The objective function helps determine the best possible design by evaluating and comparing different designs based on the specified criteria.'

But what if the lecture notes were twice as long?

In [6]:
ask_query(query="What is an objective function?", context=lecture_notes*2)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 7252 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

Here, we hit an issue: The context provided (the entire document of notes) is too large to be processed by the LLM.

One solution might be to use an LLM with a larger context window (that can process more tokens at once).

Currently, the largest context window model available is OpenAI's [GPT-4 turbo](https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo), which has a context limit of 128,000 tokens (about 90K words).

But this is still limited. 
Many documents or collections of documents can easily exceed 100K tokens.

How can we make this augmented approach more scalable?

## Introducing: Retrieval Augmented Generation (RAG)

> Retrieval Augmented Generation: Break up the reference material (the lecture notes) into smaller chunks, then find a way to pull in the relevant chunks of context based on the query.

In more detail:
- In advance:
    - Break up the reference material into chunks
    - Create an embedding of each chunk
    - Store these in an index
- During prediction:
    - Create an embedding of the query
    - Find the most similar chunk of reference material to the query by comparing embeddings
    - Pass the relevant chunk of reference material to the LLM along with the query to get an informed response

Let's start off by splitting the reference material into sentences (much smaller chunks).

In [8]:
# Split by sentence (roughly)
sentences = # TODO split the lecture notes by sentence

# Print the lecture notes split by sentence (just the first 10)
print ('\n\n--------- Sentence Break --------- \n\n'.join(sentences[:10]))

2 CHAPTER 1

--------- Sentence Break --------- 

INTRODUCTION
1.1 Introduction
Optimization is the act of achieving the best possible resul t under given circumstances.
In design, construction, maintenance, ..., engineers have to take decisions

--------- Sentence Break --------- 

The goal of all
such decisions is either to minimize eﬀort or to maximize bene ﬁt.
The eﬀort or the beneﬁt can be usually expressed as a function o f certain design variables.
Hence, optimization is the process of ﬁnding the conditions that give the maximum or the
minimum value of a function.
It is obvious that if a point x⋆corresponds to the minimum value of a function f(x), the
same point corresponds to the maximum value of the function −f(x)

--------- Sentence Break --------- 

Thus, optimization
can be taken to be minimization.
Thereis nosinglemethodavailable for solvingall optimiza tion problemseﬃciently

--------- Sentence Break --------- 

Hence,
a number of methods have been developed for solving d

### Converting our lecture notes into numbers

- We can represent a chunk of text as a point in space (anything from a single token to an entire body of text)
- Similar words should be closer together
- There are many pre-trained embedding models
- One of them is available through the OpenAI API

Here's a visualisation of what word embeddings look like:

![](images/Word%20Embeddings.png)

> Note: Anything can be turned into an embedding! Tokens, big chunks of text, images, you name it!

## Using embeddings

If you wanted to implement everything from scratch, here's what you would do:
1. Pass each chunk of text through an embedding model to get the embedding
1. Store those embeddings in a database
1. Define a function that takes in an embedding (the query) and returns you the (top n) closest (as measured by cosine distance, for example) embeddings in your database (the reference material relevant to the query)

Here, we're going to be a little more sophisticated, and use a pre-build embedding database.

## Storing and Querying our Embeddings with ChromaDB

[ChromaDB](https://www.trychroma.com/) is an open source software for storing and querying embeddings.

Let's install it.

In [2]:
!pip install chromadb

  Using cached chromadb-0.4.15-py3-none-any.whl.metadata (7.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fastapi-0.104.1-py3-none-any.whl.metadata (24 kB)
  Using cached uvicorn-0.24.0.post1-py3-none-any.whl.metadata (6.4 kB)
  Using cached posthog-3.0.2-py2.py3-none-any.whl.metadata (2.0 kB)
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
  Using cached chromadb-0.4.14-py3-none-any.whl.metadata (7.0 kB)
  Using cached chromadb-0.4.13-py3-none-any.whl.metadata (7.0 kB)
  Using cached chromadb-0.4.12-py3-none-any.whl.metadata (7.0 kB)
  Using cached pydantic-1.10.13-py3-none-any.whl.metadata (149 kB)
  Using cached fastapi-0.99.1-py3-none-any.whl.metadata (23 kB)
  Us

Once installed, we can start using the embedding database. Check out the [documentation](https://docs.trychroma.com/getting-started) for the details of what's going on here.

In [10]:
# TODO import chromadb

# Initiallize a vector store to store our text and their respective embeddings
chroma_client = # TODO create the chrome client
vector_store = # TODO create a vector store

Then we can add our chunks of text to the embedding database. 

When we do this, Chroma computes their embeddings behind the scenes, as described [here](https://docs.trychroma.com/embeddings#default-all-minilm-l6-v2).

In [11]:
# Add our sentences into the vector store (this also creates their vector embeddings behind the scenes)
# TODO add the sentences to the vector store
        # TODO pass in the sentences
        # TODO create a unique id for each sentence

Now, the vector store is ready to be queried.

In [9]:
# Querying against our own lecture notes in the vector store to get the most similar sentences to our query
# TODO query the vector store for the most similar sentences to the query
    # TODO specify the query that we want to make
    # TODO specify how many of the most similar sentences we want


NameError: name 'vector_store' is not defined

## Constraining our LLM with only lecture notes

At this point, we've managed to get the chunks of text from our reference material that are most relevant to our query. Now, we need to put both of those things in a prompt to encourage the LLM to use that reference material in its response.

In [49]:
# TODO define a function called "ask_query" that takes in a query as an argument

    # Get the most relevant sentences to our query
    context =  # TODO query the vector store for the most similar 5 results to the query
    context_list = # TODO get the most relevant sentences from the context (print to see what it looks like)
    context_string = # TODO join the sentences together into one string

    # Tell the LLM to only use the data we give it
    guide_prompt = # TODO create a prompt that contains both the context and the query
    messages = # TODO create a list of messages that ends with the quide prompt

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=100
    )

    response_content = response.choices[0].message.content

    # Give an output alongside sources
    output = f'Answer:\n\n{response_content}\n\nSources:\n\n{context_string}'
    return(output)

In [50]:
print(ask_query("What is an objective function?"))

Answer

An objective function is a function that is expressed in terms of design variables and represents the goal of either minimizing effort or maximizing benefit in decision-making. It is used in the process of optimization to find the conditions that result in the maximum or minimum value of the function.

Sources:

This criterion, wh en expressed as a function of
the design variables, is known as objective function
The goal of all
such decisions is either to minimize eﬀort or to maximize bene ﬁt.
The eﬀort or the beneﬁt can be usually expressed as a function o f certain design variables.
Hence, optimization is the process of ﬁnding the conditions that give the maximum or the
minimum value of a function.
It is obvious that if a point x⋆corresponds to the minimum value of a function f(x), the
same point corresponds to the maximum value of the function −f(x)
Howeve r, the selection of an objective
functionisnottrivial, becausewhatistheoptimal designw ithrespecttoacertaincriterion
may

# Conclusion

In this Jupyter Notebook, we explored the concept of Retrieval Augmented Generation (RAG) and how it can be used to limit the use of Language Model (LLM) to what they are best at. We started by reading a PDF file in Python and then used OpenAI's GPT-3 to ask questions about the lecture notes. We then introduced RAG and showed how it can be used to break up the reference material into smaller chunks and then find a way to pull in the relevant chunks of context based on the query. We used ChromaDB to store and query our embeddings and then constrained our LLM with only lecture notes. 

We hope this notebook has been helpful in understanding RAG and how it can be used to limit the use of LLMs to what they are best at. Feel free to modify and experiment with the code to see how it works with your own data. 
